In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_classification/imdb/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tf-nightly-2.0-preview

In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-dev20190324
GPU Enabled: False


In [0]:
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text = line.split('\t')
      text = text.split(' ')
      x = [params['word2idx'].get(w, len(word2idx)) for w in text]
      if len(x) > params['max_len']:
        x = x[:params['max_len']]
      y = int(label)
      yield x, y


def dataset(is_training, params):
  _shapes = ([None], ())
  _types = (tf.int32, tf.int32)
  _pads = (0, -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class FastText(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.embedding = tf.Variable(np.load('../vocab/word.npy'), dtype=tf.float32)
    self.out_linear = tf.keras.layers.Dense(2)
    
  def call(self, inputs):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    masks = tf.cast(tf.sign(inputs), tf.float32)
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    
    masked_mean = lambda x, mask: tf.reduce_sum(x * tf.expand_dims(mask, -1), 1) / tf.reduce_sum(mask, 1, keepdims=True)
    x = masked_mean(x, masks)
    
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
  'vocab_path': '../vocab/word.txt',
  'train_path': '../data/train.txt',
  'test_path': '../data/test.txt',
  'num_samples': 25000,
  'num_labels': 2,
  'batch_size': 32,
  'max_len': 1000,
  'num_patience': 3,
  'lr': 3e-4,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [8]:
word2idx = {}
with open(params['vocab_path']) as f:
  for i, line in enumerate(f):
    line = line.rstrip()
    word2idx[line] = i
params['word2idx'] = word2idx
params['vocab_size'] = len(word2idx) + 1


model = FastText(params)
model.build(input_shape=(None, None))
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.95)
optim = tf.keras.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = 0.

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for texts, labels in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model(texts)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
      loss = tf.reduce_mean(loss)

    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for texts, labels in dataset(is_training=False, params=params):
    logits = model(texts)
    y_pred = tf.argmax(logits, axis=-1)
    m.update_state(y_true=labels, y_pred=y_pred)
    
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))  
  history_acc.append(acc)
  
  if acc > best_acc:
    best_acc = acc
    # you can save model here
  logger.info("Best Accuracy: {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0325 05:40:48.209935 139718610032512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:421: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('dense/kernel:0', TensorShape([300, 2])),
 ('dense/bias:0', TensorShape([2])),
 ('Variable:0', TensorShape([20599, 300]))]
Reading ../data/train.txt


I0325 05:40:58.481653 139718610032512 interactiveshell.py:2882] Step 0 | Loss: 0.6701 | Spent: 10.3 secs | LR: 0.000300
I0325 05:41:09.558297 139718610032512 interactiveshell.py:2882] Step 50 | Loss: 0.6668 | Spent: 11.1 secs | LR: 0.000299
I0325 05:41:19.896712 139718610032512 interactiveshell.py:2882] Step 100 | Loss: 0.6724 | Spent: 10.3 secs | LR: 0.000298
I0325 05:41:30.766979 139718610032512 interactiveshell.py:2882] Step 150 | Loss: 0.6490 | Spent: 10.9 secs | LR: 0.000298
I0325 05:41:41.175082 139718610032512 interactiveshell.py:2882] Step 200 | Loss: 0.6674 | Spent: 10.4 secs | LR: 0.000297
I0325 05:41:51.983973 139718610032512 interactiveshell.py:2882] Step 250 | Loss: 0.6279 | Spent: 10.8 secs | LR: 0.000296
I0325 05:42:03.113948 139718610032512 interactiveshell.py:2882] Step 300 | Loss: 0.6155 | Spent: 11.1 secs | LR: 0.000295
I0325 05:42:15.285057 139718610032512 interactiveshell.py:2882] Step 350 | Loss: 0.6192 | Spent: 12.2 secs | LR: 0.000295
I0325 05:42:26.869291 13971

Reading ../data/test.txt


I0325 05:44:06.186377 139718610032512 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.820
I0325 05:44:06.187939 139718610032512 interactiveshell.py:2882] Best Accuracy: 0.820


...

I0325 06:18:37.457513 139718610032512 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.889
I0325 06:18:37.459676 139718610032512 interactiveshell.py:2882] Best Accuracy: 0.891


Reading ../data/train.txt


I0325 06:18:50.904248 139718610032512 interactiveshell.py:2882] Step 9400 | Loss: 0.0863 | Spent: 38.5 secs | LR: 0.000185
I0325 06:19:01.613096 139718610032512 interactiveshell.py:2882] Step 9450 | Loss: 0.0424 | Spent: 10.7 secs | LR: 0.000185
I0325 06:19:12.034677 139718610032512 interactiveshell.py:2882] Step 9500 | Loss: 0.0738 | Spent: 10.4 secs | LR: 0.000184
I0325 06:19:23.019459 139718610032512 interactiveshell.py:2882] Step 9550 | Loss: 0.0799 | Spent: 11.0 secs | LR: 0.000184
I0325 06:19:32.859451 139718610032512 interactiveshell.py:2882] Step 9600 | Loss: 0.1087 | Spent: 9.8 secs | LR: 0.000183
I0325 06:19:42.338783 139718610032512 interactiveshell.py:2882] Step 9650 | Loss: 0.1658 | Spent: 9.5 secs | LR: 0.000183
I0325 06:19:52.775578 139718610032512 interactiveshell.py:2882] Step 9700 | Loss: 0.1610 | Spent: 10.4 secs | LR: 0.000182
I0325 06:20:02.830017 139718610032512 interactiveshell.py:2882] Step 9750 | Loss: 0.1023 | Spent: 10.1 secs | LR: 0.000182
I0325 06:20:13.001

Reading ../data/test.txt


I0325 06:21:47.600651 139718610032512 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.889
I0325 06:21:47.602356 139718610032512 interactiveshell.py:2882] Best Accuracy: 0.891
I0325 06:21:47.608852 139718610032512 interactiveshell.py:2882] Testing Accuracy not improved over 3 epochs, Early Stop
